In [2]:
import pandas as pd
import numpy as np
import matplotlib
pd.set_option('display.max_rows', 500)

%matplotlib inline
import warnings
warnings.simplefilter('ignore', DeprecationWarning)

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import RobustScaler
from sklearn.cross_validation import LabelKFold
from scipy import interp
from sklearn import cross_validation
import pylab as pl

from sklearn import svm
from sklearn.metrics import roc_curve, auc
from os import environ
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  

C:\Users\tmeshorer\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\tmeshorer\Anaconda2\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [6]:
import os
dfs = []
for i in os.listdir("C:\\temp\\dialogs_csv\\"):
    df = pd.read_csv("C:\\temp\\dialogs_csv\\"+i)
    dfs.append(df)
all_df = pd.concat(dfs)
all_df.to_csv("../data/raw.csv")



In [8]:
df = all_df.copy()

In [9]:
len(df)

62065

In [10]:
dact_name = df[['niteType','swbdType']]
gb = dact_name.groupby(['swbdType','niteType']).count()
gb


Empty DataFrame
Columns: []
Index: [(", other), (%, uninterp), (%-, abandon), (%@, excluded), (+, abandon), (+, affirm), (+, agree), (+, ans_dispref), (+, answer), (+, apprec), (+, backchannel), (+, backchannel_q), (+, close), (+, commit), (+, completion), (+, decl_q), (+, directive), (+, excluded), (+, hedge), (+, hold), (+, neg), (+, open_q), (+, opinion), (+, or), (+, other), (+, quote), (+, reject), (+, repeat), (+, repeat_q), (+, rhet_q), (+, statement), (+, sum), (+, thank), (+, third_pty), (+, uninterp), (+, wh_q), (+, yn_decl_q), (+, yn_q), (+*, opinion), (+*, statement), (+@, excluded), (^2, completion), (^2*, completion), (^2@, excluded), (^g, tag_q), (^h, hold), (^h@, excluded), (^q, quote), (^q@, excluded), (aa, agree), (aa*, agree), (aa@, excluded), (aa^2, agree), (aa^m, agree), (aa^r, agree), (aap, maybe), (aap^r, maybe), (ad, directive), (ad(^q), directive), (ad@, excluded), (ad^c, directive), (ad^t, directive), (am, maybe), (am^r, maybe), (ar, reject), (ar^r, reject), (arp, ans_dispref), (b, backchannel), (b@, excluded), (b^m, repeat), (b^m@, excluded), (b^m^r, repeat), (b^r, backchannel), (ba, apprec), (ba,fe, apprec), (ba@, excluded), (ba^m, apprec), (ba^r, apprec), (bc, other), (bd, downplay), (bd^r, downplay), (bf, sum), (bf*, sum), (bf^g, sum), (bf^m, sum), (bf^r, sum), (bf^t, sum), (bh, backchannel_q), (bh@, excluded), (bh^r, backchannel_q), (bk, acknowledge), (bk@, excluded), (bk^m, acknowledge), (bk^r, acknowledge), (bk^t, acknowledge), (br, repeat_q), (br^m, repeat_q), (br^r, repeat_q), (by, other), (cc, commit), ...]

In [11]:
len(df3)

NameError: name 'df3' is not defined

In [12]:
df3 =df.dropna()
df3.head()

,Unnamed: 0,avg_turn_secs,avg_turn_words,dialog,end,href,id,niteType,p_da,p_speaker,...,start,swbdType,tend,tid,time_control,time_sofar,tstart,words,words_control,words_sofar
3,3,10.050000,27.0,sw2005,21.980875,sw2005.A.dialAct.xml#id(da4),da4,agree,sv,B,...,21.540875,aa,22.677375,t3,50.352909,0.440000,21.540875,1,36.0,1
4,4,9.909125,46.0,sw2005,22.677375,sw2005.A.dialAct.xml#id(da5),da5,agree,aa,A,...,22.350875,aa,34.899000,t4,49.647091,0.458375,22.219000,2,63.0,16
5,5,9.909125,46.0,sw2005,26.327500,sw2005.B.dialAct.xml#id(da6),da6,statement,aa,A,...,22.219000,sd,34.899000,t4,49.647091,4.108500,22.219000,14,63.0,14
6,6,9.909125,46.0,sw2005,27.369000,sw2005.B.dialAct.xml#id(da7),da7,abandon,sd,B,...,26.692875,%-,34.899000,t4,49.647091,5.150000,22.219000,2,63.0,18
7,7,9.909125,46.0,sw2005,29.699000,sw2005.B.dialAct.xml#id(da8),da8,statement,%-,B,...,27.449875,sd,34.899000,t4,49.647091,7.480000,22.219000,9,63.0,27


In [13]:
#remove the dates

s4 = df3['speaker'] != df3['p_speaker']
df3['tchange'] = s4
df3.tchange = df3.tchange.astype(int)
len(df3)

C:\Users\tmeshorer\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\tmeshorer\Anaconda2\lib\site-packages\pandas\core\generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


60597

In [14]:
dact_map = {
 'sd'  : 'SD',
 'b'   : 'B',
'sv'   : 'SV',
'aa'   : 'AA',
'%'    : '%' ,
'ba'   : 'SD',
'qy'   : 'Q',
'x'    : 'B',
'ny'   : 'A',
'fc'   : 'NA',
'uh'   : 'B',
'qw'   : 'Q',
'nn'   : 'A',
'bk'   : 'B',
'h'    : 'SD',
'qy^d' : 'Q',
'fw'   : 'Other',
'bh'   : 'B',
'^q'   : 'SD',
'bf'   : 'SD',
'ny^e' : 'A',
'ad' : 'SV',
'^2' : 'NA',
'b^m' : 'NA',
'qo' : 'Q',
'qh' : 'Q',
'^h' : 'A',
'ar' : 'A',
'nn^e' : 'A',
'ng' : 'A',
'br' : 'SD',
'no' : 'A',
'fp' : 'NA',
'qrr' : 'Q',
'arp' :'A',
't3' :'NA',
'oo' : 'NA',
'cc' : 'NA',
'co' : 'NA',
't1' : 'SD',
'bd' : 'SD',
'aap' : 'A',
'am'  : 'A' , 
'^g' : 'Q',
'qw^d': 'Q',
'fa' :'SD',
'%-' : '%', 
'aa^r' :'AA', 
'+@' : '+', 
'b^r':'B', 
'fe' : 'NA', 
'sd@' : 'SD', 
'o' :'NA', 
'sd(^q)' :'SD', 
'qy^t':'Q', 
'na^t' :'NA', 
'sd^e' : 'SD', 
'qr' : 'Q',
'sd^r' : 'SD', 
'na' : 'NA', 
'^h@':'NA', 
'sd^t' : 'SD', 
'fc^m' : 'NA', 
'qy^g^t' : 'Q', 
'br^m' : 'Q', 
'sv^r' : 'SV',
'sv^t' :'SV', 
'sd^c' : 'SD', 
'qy^h' : 'Q', 
'qw^t' : 'Q', 
'qy^g' : 'Q', 
'o@' :'NA', 
'ad^t' : 'AD', 
'nd' :'A', 
'sd^m' :'SD',
'"' : 'NA', 
'sv^e' :'SV', 
'sv(^q)' : 'SV', 
'%@' : '%', 
'ba^r' :'B', 
'^q@' :'Q', 
'qy^d^r' : 'Q', 
'nd^t': 'A',
't1^t' : 'NA', 
'qw^h' : 'Q', 
'ny^r' :'A', 
'^2@' : 'NA', 
'qy@' : 'Q', 
'ar^r' : 'B', 
'bk^r' :'B', 
'Other':'NA',
'nn^r' : 'NA', 
'b@':'B', 
'o^r' : 'NA', 
'bh^r' :'B', 
'bc' :'B', 
'qo^t' : 'Q', 
'sd*' : 'SD', 
'sv@' : 'SV', 
'sv^m' : 'SV',
'qy^m' : 'Q', 
'b^m@' :'B', 
'bh^m' :'B', 
'qy^c' : 'Q', 
'sv^c' : 'SV', 
'qy^c^r' : 'Q', 
'qw^d^c' : 'Q',
'qy^g^c' : 'Q', 
'qy^d@'  : 'Q', 
'ft':'NA', 
'sd^e^m' : 'SD', 
'aa^2' :'AA', 
'aa^m' :'AA', 
'qr^t' :'Q', 
'fc^t' :'NA',
'+*' :'+', 
'bk^m':'B' , 
'fc^r' : 'NA', 
'qy^r' : 'Q', 
'na^m' :'A', 
'sv^2' : 'SV', 
'ba@' : 'B', 
'sd^2' :'SD', 
'aa@' : 'AA',
'na^r' : 'NA', 
'sd(^q)^t' : 'SD', 
'na@' : 'NA', 
'qrr^t' : 'Q', 
'sd^e^t' : 'SD', 
'by' : 'B', 
'ad(^q)': 'AD', 
'no@' : 'A',
'ft^m' : 'NA', 
'qy^2' : 'Q',    
'Other' : 'NA',
'sd(^q)*' : 'SD', 
'qh^c' : 'Q',
'am^r' : 'A', 
'bf^t' : 'B', 
'br^r' : 'B', 
'Other' : 'NA', 
'qw^r' :'Q', 
'ad^c' : 'AD', 
'sd^e@' : 'SD',
'ng^m'  : 'A', 
'qr^d' : 'Q', 
'bf^g' : 'B', 
'nn^m' : 'B', 
'ft^t' : 'NA', 
'qh^g' : 'Q',   
'aap^r': 'A',
'ad@':'A',
'bf*':'B',
'bf^r':'B',
'bh@': 'B',
'bk@': 'B',
'fa^t':'NA' ,
'fp^m':'NA',
'ny^t':'B',
'qh^h':'Q',
'qh^m':'Q',
'qy^d^h':'Q',
'aa*':'AA',
'ba,fe':'B',
'ba^m':'B',
'bd^r':'B',
'bf^m':'B',
'bk^t':'B',
'co^t':'B',
'fo':'NA',
'fx':'NA',
'nn^r^t':'NA',
'nn^t':'NA',
'no^t':'A',
'o*':'NA',
'o^t':'NA',
'qo^d':'Q',
'qy^d^t':'Q',  
'^2*':'NA',
'b^m^r':'B',
'fo^c':'NA',
'o^c':'NA',
'qrr^d':'Q',
'qw^d^t':'Q',
'sd^e^r':'SD'    
}
#replace my da
s_da = df3['swbdType']
new_s_da = s_da.replace(dact_map)
df3['da'] = new_s_da

s_da = df3['p_da']
new_s_da = s_da.replace(dact_map)
df3['p_da'] = new_s_da


len(df3)

C:\Users\tmeshorer\Anaconda2\lib\site-packages\ipykernel\__main__.py:191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\tmeshorer\Anaconda2\lib\site-packages\ipykernel\__main__.py:195: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


60597

In [21]:
len(df3)

60597

In [15]:
list(df3)

['Unnamed: 0',
 'avg_turn_secs',
 'avg_turn_words',
 'dialog',
 'end',
 'href',
 'id',
 'niteType',
 'p_da',
 'p_speaker',
 'precent_secs_sofar',
 'precent_words_sofar',
 'secs',
 'speaker',
 'start',
 'swbdType',
 'tend',
 'tid',
 'time_control',
 'time_sofar',
 'tstart',
 'words',
 'words_control',
 'words_sofar',
 'tchange',
 'da']

In [16]:
df4 = df3[['dialog','speaker','tid','id','p_da','da','secs','words','tchange','tstart','tend','start','end',
 'precent_secs_sofar',
 'precent_words_sofar',
 'time_control',
 'words_control'
 ]]


df4.head()
len(df4)

60597

In [17]:
df5 = df4.copy()
df5.da = df5.da.str.strip(' ')
df5.p_da = df5.p_da.str.strip(' ')
df6 = df5[df5.da !='Other'] 
df7 = df6[df6.da !='AD'] 
df8 = df7[df7.p_da != 'Other']
df9 = df8[df8.p_da != 'AD']
df9.to_csv("../data/raw.csv")

##Prepare data before training

In [18]:
df_data = df9[['dialog','id','speaker','tchange','precent_secs_sofar','precent_words_sofar','time_control', 'words_control','start']]
df_data.head()
len(df_data)

60565

In [19]:
act_dummmies = pd.get_dummies(df9.da, prefix='Act')
p_act_dummies = pd.get_dummies(df9.p_da,prefix="PAct")
act_dummmies.head()
list(act_dummmies)

['Act_%',
 'Act_+',
 'Act_A',
 'Act_AA',
 'Act_B',
 'Act_NA',
 'Act_Q',
 'Act_SD',
 'Act_SV']

In [20]:
data = pd.concat([df_data,act_dummmies,p_act_dummies],axis=1)
data.head()
len(data)

60565

In [21]:
list(data)

['dialog',
 'id',
 'speaker',
 'tchange',
 'precent_secs_sofar',
 'precent_words_sofar',
 'time_control',
 'words_control',
 'start',
 'Act_%',
 'Act_+',
 'Act_A',
 'Act_AA',
 'Act_B',
 'Act_NA',
 'Act_Q',
 'Act_SD',
 'Act_SV',
 'PAct_%',
 'PAct_+',
 'PAct_A',
 'PAct_AA',
 'PAct_B',
 'PAct_NA',
 'PAct_Q',
 'PAct_SD',
 'PAct_SV']

In [22]:
from sklearn.preprocessing import RobustScaler

def scale(diag_name,main_df) :
    main_df_for_dialog = main_df[main_df.dialog == diag_name]
    df_speaker_A = main_df_for_dialog[main_df_for_dialog.speaker == 'A']
    df_speaker_B = main_df_for_dialog[main_df_for_dialog.speaker == 'B']
    
    # -------------- for A 
    
    #precent secs so far
    scaler = RobustScaler()
    scaler.fit(df_speaker_A.precent_secs_sofar.reshape(-1,1))
    df_speaker_A.precent_secs_sofar = scaler.transform(df_speaker_A.precent_secs_sofar.reshape(-1,1))
    #precent words so far
    scaler = RobustScaler()
    scaler.fit(df_speaker_A.precent_words_sofar.reshape(-1,1))
    df_speaker_A.precent_words_sofar = scaler.transform(df_speaker_A.precent_words_sofar.reshape(-1,1))
    #time control
    scaler = RobustScaler()
    scaler.fit(df_speaker_A.time_control.reshape(-1,1))
    df_speaker_A.time_control = scaler.transform(df_speaker_A.time_control.reshape(-1,1))
    #precent words so far
    scaler = RobustScaler()
    scaler.fit(df_speaker_A.words_control.reshape(-1,1))
    df_speaker_A.words_control = scaler.transform(df_speaker_A.words_control.reshape(-1,1))
    
    # -------------- for B 
    
    #precent secs so far
    scaler = RobustScaler()
    scaler.fit(df_speaker_B.precent_secs_sofar.reshape(-1,1))
    df_speaker_B.precent_secs_sofar = scaler.transform(df_speaker_B.precent_secs_sofar.reshape(-1,1))
    #precent words so far
    scaler = RobustScaler()
    scaler.fit(df_speaker_B.precent_words_sofar.reshape(-1,1))
    df_speaker_B.precent_words_sofar = scaler.transform(df_speaker_B.precent_words_sofar.reshape(-1,1))
    #time control
    scaler = RobustScaler()
    scaler.fit(df_speaker_B.time_control.reshape(-1,1))
    df_speaker_B.time_control = scaler.transform(df_speaker_B.time_control.reshape(-1,1))
    #precent words so far
    scaler = RobustScaler()
    scaler.fit(df_speaker_B.words_control.reshape(-1,1))
    df_speaker_B.words_control = scaler.transform(df_speaker_B.words_control.reshape(-1,1))
    
    frames = [df_speaker_A, df_speaker_B]
    return pd.concat(frames)

df_dialogs = data[['dialog','tchange']]
gb = df_dialogs.groupby('dialog').count()
gb.head()
gb2 = gb.reset_index()
after = []
for index,row in gb2.iterrows(): #This is where I'd like to go through rows of a series
        diag_name = row['dialog']
        new_df = scale(diag_name,data.copy())
        after.append(new_df)
        
       

df_both = pd.concat(after)

C:\Users\tmeshorer\Anaconda2\lib\site-packages\ipykernel\__main__.py:12: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
C:\Users\tmeshorer\Anaconda2\lib\site-packages\ipykernel\__main__.py:13: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
C:\Users\tmeshorer\Anaconda2\lib\site-packages\ipykernel\__main__.py:16: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
C:\Users\tmeshorer\Anaconda2\lib\site-packages\ipykernel\__main__.py:17: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
C:\Users\tmeshorer\Anaconda2\lib\site-packages\ipykernel\__main__.py:20: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
C:\Users\tmeshorer\Anaconda2\lib\site-packages\ipy

In [23]:
df_both['hash'] = df_both['dialog'].apply(hash)
df_both.head()
len(df_both)

60565

In [24]:
df_both.to_csv("../data/ml.csv")